In [1]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

#Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines

import statistics

from scipy.stats import rv_continuous
from scipy.stats import *

from statistics import mean
from sklearn.cluster import KMeans
from scipy.spatial import distance


######### Instance Transfer repositories ####################
from adapt.instance_based import TwoStageTrAdaBoostR2

print("Repositories uploaded!!")

Repositories uploaded!!


In [2]:
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Second Upload Completed!!")

Second Upload Completed!!


In [3]:
####### Importing libraries for map plots #######

import geopandas as gpd
import json
import altair as alt
import pandas as pd

from os import listdir
from os.path import isfile, join
import glob

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
import seaborn as sns

In [4]:
### Reading the US 2018 dataset
US_new_source = pd.read_csv('US_data/new_US_dataset/source_test.csv')
US_new_target = pd.read_csv('US_data/new_US_dataset/target_test.csv')

In [5]:
US_new_target.shape, US_new_source.shape 
US_new_target.columns

Index(['ID', 'Date', 'PM25', 'Region', 'CF_CLDTT', 'CF_PS', 'CF_Q', 'CF_Q10M',
       'CF_Q2M', 'CF_RH', 'CF_SLP', 'CF_T', 'CF_T10M', 'CF_T2M', 'CF_TPREC',
       'CF_TROPPB', 'CF_TS', 'CF_U', 'CF_U10M', 'CF_U2M', 'CF_V', 'CF_V10M',
       'CF_V2M', 'CF_ZL', 'CF_ZPBL', 'CF_PM25', 'CF_CO', 'CF_NO2', 'CF_O3',
       'CF_SO2', 'LC_Shrubs', 'LC_Herbaceous', 'LC_Agriculture', 'LC_Urban',
       'LC_Bare', 'LC_Snow', 'LC_Water', 'LC_Wetland', 'LC_Lichen',
       'LC_Closed_Forest', 'LC_Open_Forest', 'LC_Ocean', 'Elevation',
       'Dist_Primary', 'Dist_Secondary', 'Pop', 'Lon', 'Lat'],
      dtype='object')

In [6]:
######## Split into features and output ########
output_US = ['PM25']

US_y_source_df = US_new_source['PM25']
US_X_source_df = US_new_source.drop(output_US, axis = 1)

US_y_target_df = US_new_target['PM25']
US_X_target_df = US_new_target.drop(output_US, axis = 1)


######## Drop the unwanted features ########
drop_features = ['ID', 'Date', 'Region', 'Lon', 'Lat']

US_X_source_df = US_X_source_df.drop(drop_features, axis = 1)
US_X_target_df = US_X_target_df.drop(drop_features, axis = 1)

US_X_source_df.shape, US_X_target_df.shape
US_X_source_df.columns

Index(['CF_CLDTT', 'CF_PS', 'CF_Q', 'CF_Q10M', 'CF_Q2M', 'CF_RH', 'CF_SLP',
       'CF_T', 'CF_T10M', 'CF_T2M', 'CF_TPREC', 'CF_TROPPB', 'CF_TS', 'CF_U',
       'CF_U10M', 'CF_U2M', 'CF_V', 'CF_V10M', 'CF_V2M', 'CF_ZL', 'CF_ZPBL',
       'CF_PM25', 'CF_CO', 'CF_NO2', 'CF_O3', 'CF_SO2', 'LC_Shrubs',
       'LC_Herbaceous', 'LC_Agriculture', 'LC_Urban', 'LC_Bare', 'LC_Snow',
       'LC_Water', 'LC_Wetland', 'LC_Lichen', 'LC_Closed_Forest',
       'LC_Open_Forest', 'LC_Ocean', 'Elevation', 'Dist_Primary',
       'Dist_Secondary', 'Pop'],
      dtype='object')

In [7]:
######## Normalize the dataset ########
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

X_target = US_X_target_df.values 
X_target_scaled = min_max_scaler.fit_transform(X_target)
US_X_target_df = pd.DataFrame(X_target_scaled)

X_source = US_X_source_df.values 
X_source_scaled = min_max_scaler.fit_transform(X_source)
US_X_source_df = pd.DataFrame(X_source_scaled)

In [8]:
######## Split into Train-Test ########

from sklearn.model_selection import train_test_split

US_X_target_df, US_X_test_df, US_y_target_df, US_y_test_df = train_test_split(US_X_target_df, US_y_target_df, test_size = 0.30)

######## Merging the datasets ########
US_X_df = pd.concat([US_X_target_df, US_X_source_df], ignore_index=True)
US_y_df = pd.concat([US_y_target_df, US_y_source_df], ignore_index=True)

US_np_train_X = US_X_df.to_numpy()
US_np_train_y = US_y_df.to_numpy()

US_np_test_X = US_X_test_df.to_numpy()
US_np_test_y = US_y_test_df.to_numpy()

US_np_train_y_list = US_np_train_y.ravel()
US_np_test_y_list = US_np_test_y.ravel()

src_size_US = len(US_y_source_df)
tgt_size_US = len(US_y_target_df)

print("---------------------------")

---------------------------


In [10]:
####################### LOTRBoost.R2 #########################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(US_X_target_df), len(US_X_source_df)]
# sample_size = [153, 900]

n_estimators = 50
steps = 20
fold = 10
random_state = np.random.RandomState(1)

print("LOTRBoost.R2 AS")
print("-------------------------------------------")

r2scorelist_stradaboost_us = []
rmselist_stradaboost_us = []


for x in range(0, 10):

    model_lotrboost_us = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                          n_estimators = n_estimators, sample_size = sample_size,
                          steps = steps, fold = fold,
                          random_state = random_state)

    model_lotrboost_us.fit(US_np_train_X, US_np_train_y_list)
    y_pred_us = model_lotrboost_us.predict(US_np_test_X) 

    rmse_lotrboost_us = sqrt(mean_squared_error(US_np_test_y_list, y_pred_lotrboost_US))
    rmselist_lotrboost_us.append(rmse_lotrboost_US)
    print("RMSE:", rmse_lotrboost_us)

    r2_score_lotrboost_us = pearsonr(US_np_test_y_list, y_pred_lotrboost_US)
    r2_score_lotrboost_us = (r2_score_lotrboost_us[0])**2
    r2scorelist_lotrboost_us.append(r2_score_lotrboost_us)
    print("R-squared Score:", r2_score_lotrboost_us)
    
    
print("Mean RMSE of LOTRBoost:", statistics.mean(rmselist_lotrboost_us))
print("Mean R-squared of LOTRBoost:", statistics.mean(r2scorelist_lotrboost_us))
print("\n")
print("RMSE of LOTRBoost:", rmselist_lotrboost_us)
print("R-squared of LOTRBoost:", r2scorelist_lotrboost_us)


print("-------------------------------------------")

LOTRBoost.R2 AS
-------------------------------------------
Inside STrAdaBoost.R2


KeyboardInterrupt: 